# 验证 `hamiltonian.py` 模块

本 Notebook 用于测试 `DuffingOscillatorModel` 类的核心功能：
1.  验证主构造函数（基于光谱参数 `omega_q`, `anh`）能否成功创建实例。
2.  验证替代构造函数 `from_circuit_params`（基于电路参数 `e_j`, `e_c`）能否成功创建实例并正确计算有效参数。
3.  检视 `get_rwa_hamiltonian_template` 和 `get_lab_frame_hamiltonian_template` 方法返回的 QuTiP 算符是否符合预期。

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
# 导入我们自己库中的类
from quantum_sim import DuffingOscillatorModel
import matplotlib.pyplot as plt


# 配置 numpy 打印选项，使其更美观
np.set_printoptions(precision=3, suppress=True)

print("模块导入成功。")

## 1. 定义基础物理参数

我们定义一套基础的电路参数，并由此计算出光谱参数。这将用于后续两种初始化方式的验证。
单位系统：时间(ns)，角频率(rad/ns)。

In [ ]:
# --- 基础电路参数 (rad/ns) ---
# 参考 Chen, 2018, Table 2.1 (E_c=240MHz, E_J=20GHz)
# E_c / h = 240 MHz = 0.24 GHz
# E_j / h = 20 GHz
e_c_rad_ns = 0.24 * 2 * np.pi  # 转换为 rad/ns
e_j_rad_ns = 20.0 * 2 * np.pi  # 转换为 rad/ns
d_levels = 3 # 模拟的能级数

# --- 由此计算出的有效光谱参数 (rad/ns) ---
omega_q_rad_ns = np.sqrt(8 * e_j_rad_ns * e_c_rad_ns) - e_c_rad_ns
anh_rad_ns = -e_c_rad_ns

print(f"基础电路参数 (rad/ns): E_j = {e_j_rad_ns:.3f}, E_c = {e_c_rad_ns:.3f}")
print(f"计算得到的光谱参数 (rad/ns): ω_q = {omega_q_rad_ns:.3f}, η = {anh_rad_ns:.3f}")
print(f'换算为 (GHz): E_j = {e_j_rad_ns / (2 * np.pi):.3f}, E_c = {e_c_rad_ns / (2 * np.pi):.3f}')
print(f'换算为 (GHz): ω_q = {omega_q_rad_ns / (2 * np.pi):.3f}, η = {anh_rad_ns / (2 * np.pi):.3f}')

## 2. 验证两种构造函数

我们将分别使用光谱参数和电路参数来创建 `DuffingOscillatorModel` 实例，并对比它们的属性，以验证两种初始化方式的等效性。

In [ ]:
# 方式一：直接使用光谱参数初始化
model_from_spec = DuffingOscillatorModel(d=d_levels, omega_q=omega_q_rad_ns, anh=anh_rad_ns)
print("--- 通过光谱参数初始化成功 ---")
print(f"  实例中的 ω_q: {model_from_spec.omega_q:.3f} GHz")
print(f"  实例中的 η: {model_from_spec.anh:.3f} GHz")
print("-" * 30)

# 方式二：使用电路参数通过替代构造函数初始化
model_from_circuit = DuffingOscillatorModel.from_circuit_params(d=d_levels, e_j=e_j_rad_ns, e_c=e_c_rad_ns)
print("--- 通过电路参数初始化成功 ---")
print(f"  实例中的 ω_q (内部计算): {model_from_circuit.omega_q:.3f} GHz")
print(f"  实例中的 η (内部计算): {model_from_circuit.anh:.3f} GHz")
print("-" * 30)

# 验证二者计算结果是否一致
assert np.isclose(model_from_spec.omega_q, model_from_circuit.omega_q), "ω_q 不一致！"
assert np.isclose(model_from_spec.anh, model_from_circuit.anh), "非谐性 η 不一致！"

print("✅ 验证通过：两种初始化方式结果一致。")

## 3. 验证哈密顿量模板

现在我们从一个已创建的实例中，获取 RWA 和 Lab Frame 的哈密顿量模板，并打印出来检视其结构和数值是否正确。

In [ ]:
# 获取 RWA 哈密顿量模板
drift_hamiltonian = model_from_spec.get_drift_hamiltonian(use_rwa=True)
control_hamiltonians = model_from_spec.get_control_hamiltonian(use_rwa=True)

h_control_i = control_hamiltonians['XY_drive']['H_I']
h_control_q = control_hamiltonians['XY_drive']['H_Q']

print("--- RWA 哈密顿量模板 ---")
print("\n1. 自由哈密顿量 (H_drift_rot):")
print(drift_hamiltonian)
print("\n2. I 路控制算符 (H_control_I, a + a†):")
print(h_control_i)
print("\n3. Q 路控制算符 (H_control_Q, i(a† - a)):")
print(h_control_q)

In [ ]:
# 获取 Lab Frame 哈密顿量模板
drift_hamiltonian = model_from_spec.get_drift_hamiltonian(use_rwa=False)
control_hamiltonians = model_from_spec.get_control_hamiltonian(use_rwa=False)

print("--- Lab Frame 哈密顿量模板 ---")
print("\n1. 自由哈密顿量 (H_drift_lab):")
print(drift_hamiltonian)
print("\n2. 控制算符 (H_control_lab, a + a†):")
print(control_hamiltonians)

## 4. 验证哈密顿量画能级图功能



In [ ]:
# --- 验证能级图可视化功能 ---

print("正在为 model_from_spec 对象生成能级图...")

# 直接从模型对象调用新的可视化方法
model_from_spec.visualize_energy_levels(unit='GHz')

print("\n能级图（单位 GHz）显示完成。")


print("\n尝试使用 rad/ns 单位重新生成...")
model_from_spec.visualize_energy_levels(unit='rad/ns')
print("\n能级图（单位 rad/ns）显示完成。")